In [1]:
import mysql.connector
from selenium import webdriver            
import time
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup

In [2]:
driver=webdriver.Firefox()

In [3]:
amazon_urls = [
    "https://www.amazon.in/stores/page/C449E603-EFEE-4862-9271-05AB5EBEEE6A?",
    "https://www.amazon.in/stores/page/88A144B6-20D9-4EB9-B8B0-D73D6DB9ADF5?",
    "https://www.amazon.in/stores/page/4198D013-CE02-49DC-892D-8D7D79D8E65E?",
    "https://www.amazon.in/stores/page/DE47F10C-F6E8-4B2C-A5A6-53555256878B?",
    "https://www.amazon.in/stores/page/6F965F90-19EC-4EC7-A186-400CF6426D64?",
    "https://www.amazon.in/stores/page/7482736C-3322-4AB1-AC32-65E32EE05CE1?",
    "https://www.amazon.in/stores/page/956732E6-CB59-49E6-AE32-62B66C5C63AB?",
    "https://www.amazon.in/stores/page/38C47895-90A2-46B8-A861-4BAF09AF0467?",
    "https://www.amazon.in/stores/page/AC015DF5-9C88-4F5D-8F1F-E41E88C30567?",
    "https://www.amazon.in/stores/page/DB5BEC81-8229-49D1-B582-EC50B62BD4E5?"
]


In [4]:
def insert_into_database(data):
    db_connection = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "Qwerty@12345",
        database = "data"
    )

    cursor = db_connection.cursor()

    for index, row in data.iterrows():
        title = row['title']
        price = row['price']
        url = row['url']
        sql = "INSERT INTO products (title, price, url) VALUES (%s, %s, %s)"
        val = (title, price, url)
        cursor.execute(sql, val)
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

In [5]:

def data(soup):
    all_dicts = []
    a = soup.find_all('li', class_='ProductGridItem__itemOuter__KUtvv ProductGridItem__grid-tapestry__D0bpE ProductGridItem__fixed__DQzmO')
    
    for i in a:
        b = BeautifulSoup(str(i), 'html.parser')

        title_element = b.find('div', class_='Title__truncateTitle__DGaow').get_text(strip=True)
        price_element = b.find('span', class_='Price__whole__mQGs5')
        link = b.find('div', class_='Title__truncateTitle__DGaow').find('a').get('href')


        if price_element is not None:
            price = int(price_element.get_text(strip=True).replace(',', ''))
        else:
            price = None

        result_dict = {'title': title_element, 'price': price,'url':'https://www.amazon.in'+link}
        all_dicts.append(result_dict)

    df = pd.DataFrame(all_dicts)
    return df

In [ ]:
all_dfs = []

for j in amazon_urls:
    driver.get(j)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    result = data(soup)
    all_dfs.append(result)
    print('Success for the page: ' + j)
    time.sleep(2)


In [7]:
driver.quit()

In [8]:
df=pd.concat(all_dfs,ignore_index=True)

In [9]:
insert_into_database(df)